### 모델을 향상시키는 방법
1. 모델 바꾸기
2. 데이터 수정 및 추가
3. hyperparameter tuning

- 2번이 가장 중요

## Hyperparameter Turning
- 모델이 스스로 학습하지 않는 값은 사람이 지정(learning rate, 모델 크기, optimizer 등)
- 마지막 0.01%를 쥐어짜야 할 때 도전해볼만함
- 가장 기본적인 방법
  - grid search: 일정한 범위를 정해 값을 자름, 차례대로 학습 수행한 후 가장 좋은 성능을 내는 것 찾음
  - random search: random하게 학습 수행한 후 가 장 좋은 것 찾기
- 최근에는 베이지안 기반 기법들이 주도 (BOHB)

## Ray
- multi-node multi processing 지원 모듈
- ML/DL 병렬 처리 모듈의 표준
- Hyperparameter search를 위한 다양한 모듈 제공

In [ ]:
data_dir = os.path.abspath("./data")
load_data(data_dir)
# config에 search space 지정
config = {
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
"batch_size": tune.choice([2, 4, 8, 16])
}

# 학습 스케줄링 알고리즘 지정
scheduler = ASHAScheduler(
    metric="loss", mode="min", max_t=max_num_epochs, grace_period=1, reduction_factor=2)
reporter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"])

# 병렬 처리 양식으로 학습 시행
result = tune.run( #tune.run: 여러개의 cpu에 분산시켜줌
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
    config=config, num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter)

## Ray-tune for Hyperparameter Tuning

In [1]:
! pip install ray 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 14.2 MB/s eta 0:00:00


In [2]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 1.4 MB/s eta 0:00:00


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=be3b681fc82bac0aa34159022ccd50f29a3772e6ed9eec48a70c313d3a0b6f01
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [4]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import wandb

In [5]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset


In [6]:
class Net(nn.Module): 
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2) #마지막 layer 2개
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
# 하나의 함수로 표현되야지만 lay가 불러올 수 있음
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # entropy 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    wandb.init(project='Ray tuning hyperparameter', entity='imsmile2000')
    wandb.watch(net)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        wandb.log({"val_loss": val_loss})
        wandb.log({"loss": loss})

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [12]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [13]:
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch

def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    # 실행
    result = tune.run( 
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    # 가장 높은 성능의 trial 불러오기
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    wandb.login(key="73a1afbee8c19ff574b3531adf3a4e0961d0fbc1")
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

wandb: Currently logged in as: imsmile2000. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Files already downloaded and verified
Files already downloaded and verified
== Status ==
Current time: 2023-03-17 01:40:28 (running for 00:00:00.11)
Memory usage on this node: 3.1/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/1 GPUs, 0.0/7.39 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/train_cifar_2023-03-17_01-40-28
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+------------------+--------------+------+------+-------------+
| Trial name              | status   | loc              |   batch_size |   l1 |   l2 |          lr |
|-------------------------+----------+------------------+--------------+------+------+-------------|
| train_cifar_b22a4_00000 | RUNNING  | 172.28.0.12:4162 |           16 |   32 |   32 | 0.000414851 |
| train_cifar_b22a4_00001 | PENDING  |                  |            8 |   16 |   32 | 0.052

(func pid=4162) /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(func pid=4162)   warnings.warn(_create_warning_msg(
(func pid=4162) wandb: Currently logged in as: imsmile2000. Use `wandb login --relogin` to force relogin
(func pid=4162) wandb: Tracking run with wandb version 0.14.0
(func pid=4162) wandb: Run data is saved locally in /root/ray_results/train_cifar_2023-03-17_01-40-28/train_cifar_b22a4_00000_0_batch_size=16,l1=32,l2=32,lr=0.0004_2023-03-17_01-40-28/wandb/run-20230317_014033-x34n97a8
(func pid=4162) wandb: Run `wandb offline` to turn off syncing.
(func pid=4162) wandb: Syncing run worthy-bl

== Status ==
Current time: 2023-03-17 01:40:36 (running for 00:00:08.09)
Memory usage on this node: 3.7/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/1 GPUs, 0.0/7.39 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/train_cifar_2023-03-17_01-40-28
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+------------------+--------------+------+------+-------------+
| Trial name              | status   | loc              |   batch_size |   l1 |   l2 |          lr |
|-------------------------+----------+------------------+--------------+------+------+-------------|
| train_cifar_b22a4_00000 | RUNNING  | 172.28.0.12:4162 |           16 |   32 |   32 | 0.000414851 |
| train_cifar_b22a4_00001 | PENDING  |                  |            8 |   16 |   32 | 0.0520736   |
| train_cifar_b22a4_00002 | PENDING  |                  |          

Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_b22a4_00000,0.4578,2023-03-17_01-43-59,False,,c8a5c53c684b4164822861b71abb449a,b5c83dd34c70,6,1.49463,172.28.0.12,4162,True,208.379,31.8757,208.379,1679017439,0,,6,b22a4_00000,0.00377369


== Status ==
Current time: 2023-03-17 01:41:09 (running for 00:00:40.76)
Memory usage on this node: 3.7/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -2.299063328170776
Resources requested: 2.0/2 CPUs, 0/1 GPUs, 0.0/7.39 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/train_cifar_2023-03-17_01-40-28
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+------------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name              | status   | loc              |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-------------------------+----------+------------------+--------------+------+------+-------------+---------+------------+----------------------|
| train_cifar_b22a4_00000 | RUNNING  | 172.28.0.12:4162 |           16 |   32 |   32 | 0.000414851 | 2.29906 |     0.

2023-03-17 01:44:23,889	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-03-17 01:44:24,922	ERROR tune.py:794 -- Trials did not complete: [train_cifar_b22a4_00000, train_cifar_b22a4_00001, train_cifar_b22a4_00002, train_cifar_b22a4_00003, train_cifar_b22a4_00004, train_cifar_b22a4_00005, train_cifar_b22a4_00006, train_cifar_b22a4_00007, train_cifar_b22a4_00008, train_cifar_b22a4_00009]
2023-03-17 01:44:24,930	INFO tune.py:798 -- Total run time: 236.62 seconds (236.57 seconds for the tuning loop).
2023-03-17 01:44:24,932	WARNING tune.py:804 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


== Status ==
Current time: 2023-03-17 01:44:24 (running for 00:03:56.57)
Memory usage on this node: 3.7/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: -1.6692905458450318 | Iter 2.000: -2.099749384880066 | Iter 1.000: -2.299063328170776
Resources requested: 2.0/2 CPUs, 0/1 GPUs, 0.0/7.39 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/train_cifar_2023-03-17_01-40-28
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+------------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name              | status   | loc              |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-------------------------+----------+------------------+--------------+------+------+-------------+---------+------------+----------------------|
| train_cifar_b22a4_00000 | RUNNING  | 172.28.0.12:4162 |           16 |   32 |   32 | 0

AttributeError: ignored

## Further Question
1. 모델의 모든 layer에서 learning rate가 항상 같아야 할까요? 같이 논의해보세요!
  - 달라도 된다..! fine-tuning 할때!
2. ray tune을 이용해 hyperparameter 탐색을 하려고 합니다. 아직 어떤 hyperparmeter도 탐색한적이 없지만 시간이 없어서 1개의 hyperparameter만 탐색할 수 있다면 어떤 hyperparameter를 선택할 것 같나요? 같이 논의해보세요!
  - learning rate가 가장 중요함
  - 그 다음으로 hidden unit의 개수, mini-batch의 크기, 모멘텀 값
  - 그 다음으로 layer의 수 등..

## 회고
- hyperparameter 튜닝을 통해 향상되는 효과는 미미하다...튜닝보다 데이터 하나하나가 훨씬 중요하다!
- 성능을 올릴 목적보다는 모델을 가볍게 만드는 목적으로 쓰자